## Simulaciones consecutivas de rampas

In [ ]:
#-----Importa los paquetes escenciales

from __future__ import division
import hoomd
import hoomd.md

#-----Define las variables relevantes

p0 = 1.0;
p1 = 1.2;
p2 = 1.4;
p3 = 1.6;
p4 = 1.8;
t_max  = 5.0;

#-----Crea un contexto de simulación

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Extrae la configuración del centro de masa de las partículas

system = hoomd.init.read_gsd(".gsd",frame = -1)

#-----Define cada mesógeno en el marco de referencia local de cada centro de masa.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Declara que las moléculas creadas son cuerpos rígidos

rigid.create_bodies();

#-----Define la energía potencial

nl = hoomd.md.nlist.tree()

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode='shift')

#-----Define la interacción entre las especies de la simulación

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Selecciona un integrador standard

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define el grupo sobre el cual se integraran las ecuaciones de movimiento

mesogens = hoomd.group.rigid_center();

In [ ]:
#----Integra usando un NPT

press_1 = hoomd.variant.linear_interp(points = [(-----,p0), (-----, p1)])
press_2 = hoomd.variant.linear_interp(points = [(-----,p1), (-----, p2)])
press_3 = hoomd.variant.linear_interp(points = [(-----,p2), (-----, p3)])
press_4 = hoomd.variant.linear_interp(points = [(-----,p3), (-----, p4)])

npt = hoomd.md.integrate.npt(group = mesogens, kT = t_max, tau = 1.0, tauP = 2.5, P = press_1);
npt.randomize_velocities(seed = 42)

#-----Escribe en la información en archivos

log_file = "T_" + str(t_max) + "_P_" + str(p1) + "_ramp.log"
gsd_file = "T_" + str(t_max) + "_P_" + str(p1) + "_ramp.gsd" 
meso_gsd_file = "T_CM_" + str(t_max) + "_P_" + str(p1) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True);

#-----Imprime la densidad del sistema

system.get_metadata()